<h2>Queremos fazer uma análise de quais partidos,
mais especificamente governos de UFs que foram mais permissivos 
em relação à degradação do meio ambiente.
Para isso, precisamos de dados do meio ambiente e de eleições.

<h3>Instalando base dos dados

In [29]:
!pip install basedosdados

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


Importando ferramentas

In [30]:
!pip install plotly

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


In [3]:
import basedosdados as bd
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go

<h3>Lendo tabela de informação de candidatos eleitos

<h4> Para pegar as informações relevantes, selecionamos só os governantes de UFs que têm dados sobre sua cobertura de mata amazônia na basedosdados 
     <h4>A consulta em si está na pasta de consultas SQL com o nome 'GovernoEstado.sql'
         <h4> Aqui importamos o csv gerado pela consulta descrita acima

In [4]:
csvGov= pd.read_csv('ConsultasEmCsv/governosestados.csv')

In [15]:
csvGov.head(15)

,anos,sigla_uf,sigla_partido
0,1994,AC,PPR
1,1998,AC,PT
2,2002,AC,PT
3,2006,AC,PT
4,2010,AC,PT
5,2014,AC,PT
6,2018,AC,PP
7,1994,AM,PPR
8,1998,AM,PFL
9,2002,AM,PPS


<h3>Lendo tabela de informação de cobertura do bioma amazônia por estado e ano

<h4> Pegamos as informações da área do bioma Amazônia relacionada ao estado e ao ano em que foram registradas
     <h4>A consulta em si está na pasta de consultas SQL com o nome 'AreaTotAmazonia.sql'
         <h4> Aqui importamos o csv gerado pela consulta descrita acima

In [8]:
csvCob= pd.read_csv('ConsultasEmCsv/coberturaanoestado.csv')

In [97]:
csvCob

,ano,estado_abrev,areatotal
0,1985,AC,158629.949213
1,1986,AC,158481.673831
2,1987,AC,158058.639895
3,1988,AC,157346.219951
4,1989,AC,156778.336434
...,...,...,...
310,2015,TO,6951.741695
311,2016,TO,6592.706262
312,2017,TO,6853.481358
313,2018,TO,6645.391944


<h1> Queremos saber quais partidos foram mais permissivos em relação a perda do bioma da amazônia

<h3> Relacionando as duas tabelas para obter o dataset o qual utilizaremos para operações e análises no pandas, com o objetivo de responder a pergunta acima:
   

<h4> Com as duas tabelas acima, fazemos um inner join para relacionar as mudanças dos biomas com o partido governante
     <h4>A consulta que relaciona as tabelas está na pasta de consultas SQL com o nome 'JoinGovArea.sql'
         <h4> Abaixo importamos o csv gerado pela consulta descrita acima

In [98]:
lig = pd.read_csv('ConsultasEmCsv/coberturajoinpartidos.csv')

In [99]:
lig.head(15)

,ano,sigla_uf,areatotal,sigla_partido
0,1994,AC,1.547424e+05,PPR
1,1998,AC,1.522341e+05,PT
2,2002,AC,1.480173e+05,PT
3,2006,AC,1.477197e+05,PT
4,2010,AC,1.465175e+05,PT
5,2014,AC,1.456493e+05,PT
6,2018,AC,1.441178e+05,PP
7,1994,AM,1.505151e+06,PPR
8,1998,AM,1.503607e+06,PFL
9,2002,AM,1.503723e+06,PPS


<h3> Adicionando coluna na qual queremos armazenar a porcentagem de cobertura de amazônia perdida por mandato de governo

In [100]:
#Usando o pandas, adicionando uma coluna inicialmente vazia
lig["delta"] = None
lig.head(15)

,ano,sigla_uf,areatotal,sigla_partido,delta
0,1994,AC,1.547424e+05,PPR,None
1,1998,AC,1.522341e+05,PT,None
2,2002,AC,1.480173e+05,PT,None
3,2006,AC,1.477197e+05,PT,None
4,2010,AC,1.465175e+05,PT,None
5,2014,AC,1.456493e+05,PT,None
6,2018,AC,1.441178e+05,PP,None
7,1994,AM,1.505151e+06,PPR,None
8,1998,AM,1.503607e+06,PFL,None
9,2002,AM,1.503723e+06,PPS,None


<h3>Função para armazenar a perda de cobertura do bioma por mandato
    <h4> Nessa função, percorremos as linhas do database e pegamos a informação da área antes e após o mandato, para armazenar a porcentagem de quanto mudou na coluna delta

In [101]:
lig["delta"] = None
for index in lig.index:
    line = lig.iloc[index]
    ano = line.ano
    est = line.sigla_uf
    areaantiga = line.areatotal
    areaatual = lig[(lig.sigla_uf==est) & (lig.ano == ano + 4)].areatotal
    if len(areaatual) == 0:
        areaatual= None
    else:
        areaatual= areaatual.iloc[0]
        lig.loc[index,'delta'] = ((areaatual - areaantiga)/ areaantiga)*100

#variável 'deltadb' agora armazena os deltas de áreas
deltadb = lig
#removemos linhas em que nao temos informações do desmatamento
deltadb = deltadb.dropna()

In [102]:
deltadb

,ano,sigla_uf,areatotal,sigla_partido,delta
0,1994,AC,1.547424e+05,PPR,-1.62098
1,1998,AC,1.522341e+05,PT,-2.769942
2,2002,AC,1.480173e+05,PT,-0.201035
3,2006,AC,1.477197e+05,PT,-0.813853
4,2010,AC,1.465175e+05,PT,-0.592542
5,2014,AC,1.456493e+05,PT,-1.051536
7,1994,AM,1.505151e+06,PPR,-0.102529
8,1998,AM,1.503607e+06,PFL,0.007702
9,2002,AM,1.503723e+06,PPS,-0.401206
10,2006,AM,1.497690e+06,PMDB,-0.112286


<h4> Com a porcentagem da área perdida relacionada com o partido que estava no mandato, podemos fazer cálculos para comparar quem foi mais permissivo em relação à perda da biodiversidade

In [103]:
#Vamos usar as duas colunas, a que indica o partido que governou, e o saldo do mandato em relação a área do bioma
ptbioma= deltadb[['sigla_partido','delta'] ]
ptbioma.head(15)

,sigla_partido,delta
0,PPR,-1.62098
1,PT,-2.769942
2,PT,-0.201035
3,PT,-0.813853
4,PT,-0.592542
5,PT,-1.051536
7,PPR,-0.102529
8,PFL,0.007702
9,PPS,-0.401206
10,PMDB,-0.112286


<h3> Somamos porcentagens por cada partido e fazemos uma média em relação a quantos mandatos o partido teve, para poder comparar qual foi o mais permissivo

<h4>Somando porcentagens por partido:

In [104]:
a= ptbioma.groupby('sigla_partido').delta.sum()

<h4>Contando instâncias de partidos para fazer a média

In [105]:
b = ptbioma.groupby('sigla_partido').sigla_partido.count()

<b> Pegando a média por partido

In [106]:
mediapercentpart = a/b
mediapercentpart.sort_values(ascending=True)

sigla_partido
PPR       -6.260845
PFL       -4.571913
PPS       -3.279318
PC do B   -2.601661
PMDB      -2.042366
PSDB      -1.282231
PT        -1.156398
PDT       -1.096243
PP        -0.902800
PPB       -0.822836
PTB       -0.453156
PROS      -0.292676
PMN       -0.212082
PSL       -0.142464
PSB       -0.042159
dtype: float64

<h3>Vamos selecionar os estados que tiveram perdas mais massivas em relaçao ao bioma da amazônia

<p>Queremos a perda total no espaço amostral de tempo por cada estado, vamos selecionar o que teve mais área perdida comparativamente para visualizar os dados de como foram a sequencia de mandatos, e em qual mandato mais teve perda
   

In [107]:
perdaestado= deltadb.groupby('sigla_uf').delta.sum()

<p>Temos a informação da soma das perdas, mas ainda precisamos normalizar o dado, já que a cobertura de tempo de diferentes estados também é diferente

In [108]:
countestado=deltadb.groupby('sigla_uf').sigla_uf.count()

In [109]:
pestcpt= perdaestado/countestado
pestcpt.sort_values(ascending=True)

sigla_uf
MA   -5.146821
MT   -4.209027
RO   -4.144400
TO   -2.966543
PA   -1.764645
AC   -1.174981
RR   -0.500949
AM   -0.185513
AP   -0.064529
dtype: float64

<b> Vemos que comparativamente, as unidades federais que mais sofreram foram: Maranhão, Mato Grosso e Rondônia

<h3> Vamos explorar os dados e procurar quais foram os governos desses estados para ver qual foi mais nocivo para o bioma da amazônia

<p> Selecionando dados referentes aos 3 estados que mais sofreram com a perda do bioma amazônia e adicionando os mandatos:

In [176]:
rodb= deltadb.query('sigla_uf == "RO" or sigla_uf == "MT" or sigla_uf == "MA"')
rodb["governo_ano"]= rodb["ano"].astype(str) + "-" + ((rodb["ano"] + 4 ).astype(str))  +"("+ rodb["sigla_partido"] + ")"
rodb

,ano,sigla_uf,areatotal,sigla_partido,delta,governo_ano
21,1994,MA,75481.156639,PFL,-5.797984,1994-1998(PFL)
22,1998,MA,71104.771291,PFL,-8.05166,1998-2002(PFL)
23,2002,MA,65379.657086,PFL,-7.14347,2002-2006(PFL)
24,2006,MA,60709.280675,PDT,-4.124277,2006-2010(PDT)
25,2010,MA,58205.461653,PMDB,-3.161876,2010-2014(PMDB)
26,2014,MA,56365.076957,PC do B,-2.601661,2014-2018(PC do B)
28,1998,MT,409861.953011,PSDB,-6.03587,1998-2002(PSDB)
29,2002,MT,385123.217209,PPS,-8.91838,2002-2006(PPS)
30,2006,MT,350776.466734,PPS,-2.140185,2006-2010(PPS)
31,2010,MT,343269.200508,PMDB,0.258326,2010-2014(PMDB)


In [132]:
mtdb= rodb.query('sigla_uf == "MT"')
mtdb

,ano,sigla_uf,areatotal,sigla_partido,delta,governo_ano
28,1998,MT,409861.953011,PSDB,-6.03587,1998-2002_PSDB
29,2002,MT,385123.217209,PPS,-8.91838,2002-2006_PPS
30,2006,MT,350776.466734,PPS,-2.140185,2006-2010_PPS
31,2010,MT,343269.200508,PMDB,0.258326,2010-2014_PMDB


In [133]:
madb= rodb.query('sigla_uf == "MA"')
madb

,ano,sigla_uf,areatotal,sigla_partido,delta,governo_ano
21,1994,MA,75481.156639,PFL,-5.797984,1994-1998_PFL
22,1998,MA,71104.771291,PFL,-8.05166,1998-2002_PFL
23,2002,MA,65379.657086,PFL,-7.14347,2002-2006_PFL
24,2006,MA,60709.280675,PDT,-4.124277,2006-2010_PDT
25,2010,MA,58205.461653,PMDB,-3.161876,2010-2014_PMDB
26,2014,MA,56365.076957,PC do B,-2.601661,2014-2018_PC do B


In [210]:
rondb= rodb.query('sigla_uf == "RO"')
rondb

,ano,sigla_uf,areatotal,sigla_partido,delta,governo_ano
39,1998,RO,183489.029383,PFL,-5.065036,1998-2002(PFL)
40,2002,RO,174195.243880,PSDB,-7.892197,2002-2006(PSDB)
41,2006,RO,160447.412213,PPS,-1.6575,2006-2010(PPS)
42,2010,RO,157787.996468,PMDB,-2.020351,2010-2014(PMDB)
43,2014,RO,154600.125680,PMDB,-4.086917,2014-2018(PMDB)


<h2> Relacionando cada uma com os mandatos de partidos:
   

In [245]:
mtdb

,ano,sigla_uf,areatotal,sigla_partido,delta,governo_ano
28,1998,MT,409861.953011,PSDB,-6.03587,1998-2002_PSDB
29,2002,MT,385123.217209,PPS,-8.91838,2002-2006_PPS
30,2006,MT,350776.466734,PPS,-2.140185,2006-2010_PPS
31,2010,MT,343269.200508,PMDB,0.258326,2010-2014_PMDB


<h3>Modelando os dados para poder plotar em um gráfico:

In [259]:
col = pd.unique(rodb["sigla_uf"])

newdf = pd.DataFrame(columns = col)

newdf["ano"] = madb["ano"]
newdf["MA"] = madb["areatotal"]
pd.to_numeric(rondb['ano'])
pd.to_numeric(newdf['ano'])

newdf = newdf.merge(rondb,how = 'inner', on = 'ano').merge(mtdb,how= 'inner', on = 'ano')

newdf = newdf[['ano','MA','areatotal_x','areatotal_y']]
newdf = newdf.rename(columns = {'areatotal_x': 'RO', 'areatotal_y': 'MT'})
newdf


,ano,MA,RO,MT
0,1998,71104.771291,183489.029383,409861.953011
1,2002,65379.657086,174195.243880,385123.217209
2,2006,60709.280675,160447.412213,350776.466734
3,2010,58205.461653,157787.996468,343269.200508


In [262]:
fig = px.line(newdf, x='ano', y=newdf.columns,
             title='Área total(km²) do bioma amazônia nesses 3 estados ao longo dos anos')

fig.show()

In [260]:
df = px.data.stocks()
fig = px.line(df, x="date", y=df.columns,
              hover_data={"date": "|%B %d, %Y"},
              title='custom tick labels')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

In [169]:
df

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472
